## 3 - Medium Level Example - 1-Axis tracker by hour (gendaylit)

Example demonstrating the use of doing hourly smiulations with Radiance gendaylit for 1-axis tracking. This is a medium level example because it also explores a couple subtopics:

#### Subtopics:
<ul>
    <li> The structure of the tracker dictionary "trackerDict". </li>
    <li> How to calculate GCR </li>
    <li> How to make a cell-level module </li>
    <li> Various methods to use the trackerdictionary for analysis. </li>
</ul>
 
#### Doing full year simulations with gendaylit: 

Performs the simulation hour by hour requires either a good computer or some patience, since there are ~4000 daylight-hours in the year. With a 32GB RAM, Windows 10 i7-8700 CPU @ 3.2GHz with 6 cores this takes 1 day. The code also allows for multiple cores or HPC use -- there is documentation/examples inside the software at the moment, but that is an advanced topic. The procedure can be broken into shorter steps for one day or a single timestamp simulation which is exemplified below.

### Steps:
<ol>
    <li> <a href='#step1'> Create a folder for your simulation, and load bifacial_radiance </a></li> 
    <li> <a href='#step2'> Define all your system variables </a></li> 
    <li> <a href='#step3'> Create Radiance Object, Set Albedo and Weather </a></li> 
    <li> <a href='#step4'> Make Module: Cell Level Module Example </a></li>    
    <li> <a href='#step5'> Calculate GCR</a></li> 
    <li> <a href='#step6'> Set Tracking Angles </a></li> 
    <li> <a href='#step7'> Generate the Sky </a></li> 
    <li> <a href='#step8'> Make Scene 1axis </a></li> 
    <li> <ol type="A"><li><a href='#step9a'> Make Oct and AnalyzE 1 HOUR </a></li> 
    <li> <a href='#step9b'> Make Oct and Analye Range of Hours </a></li> 
        <li> <a href='#step9c'>  Make Oct and Analyze All Tracking Dictionary </a></li> </ol>
</ol>

And finally:  <ul> <a href='#condensed'> Condensed Version: All Tracking Dictionary </a></ul>   

<a id='step1'></a>


## 1. Create a folder for your simulation, and load bifacial_radiance 

First let's set the folder where the simulation will be saved. By default, this is the TEMP folder in the bifacial_radiance distribution.

The lines below find the location of the folder relative to this Jupyter Journal. You can alternatively point to an empty directory (it will open a load GUI Visual Interface) or specify any other directory in your computer, for example:

#### testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal3'


In [1]:
import os
testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


#### Pay attention: different importing method:

So far we've used "from bifacial_radiance import *" to import all the bifacial_radiance files into our working space in jupyter. For this journal we will do a "import bifacial_radiance" . This method of importing requires a different call for some functions as you'll see below. For example, instead of calling demo = RadianceObj(path = testfolder) as on Tutorial 2, in this case we will neeed to do demo = bifacial_radiance.RadianceObj(path = testfolder). 

In [2]:
import bifacial_radiance
import numpy as np
import pprint    # We will be pretty-printing the trackerdictionary throughout to show its structure.

<a id='step2'></a>

## 2. Define all your system variables

Just like in the condensed version show at the end of Tutorial 2, for this tutorial we will be starting all of our system variables from the beginning of the jupyter journal, instead than throughout the different cells (for the most part)

In [3]:
simulationName = 'Tutorial 3'    # For adding a simulation name when defning RadianceObj. This is optional.
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = False
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)


<a id='step3'></a>

## 3. Create Radiance Object, Set Albedo and Weather

Same steps as previous two tutorials, so condensing it into one step. You hopefully have this down by now! :)


<div class="alert alert-warning">
Notice that we are doing bifacial_radiance.RadianceObj because we change the import method for this example!
</div>

In [4]:
demo = bifacial_radiance.RadianceObj(simulationName, path = testfolder)  # Adding a simulation name. This is optional.
demo.setGround(albedo) 
epwfile = demo.getEPW(lat = lat, lon = lon) 
metdata = demo.readWeatherFile(weatherFile = epwfile) 


path = C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


<a id='step4'></a>

## 4. Make Module: Cell Level Module Example

Instead of doing a opaque, flat single-surface module, in this tutorial we will create a module made up by cells. We can define variuos parameters to make a cell-level module, such as cell size and spacing between cells. To do this, we will pass a dicitonary with the needed parameters to makeModule, as shown below.

<div class="alert alert-warning">
Since we are passing a cell-level dictionary, the values for module's x and y of the module will be calculated by the software -- no need to pass them (and if you do, they'll just get ignored)
    </div>

In [5]:
numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=moduletype, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                xgap=xgap, ygap=ygap, zgap=zgap, numpanels=numpanels, 
                cellLevelModuleParams=cellLevelModuleParams, 
                axisofrotationTorqueTube=axisofrotationTorqueTube)


Module Name: Custom_Cell-Level_Module
REWRITING pre-existing module file. 
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module Custom Cell-Level Module successfully created


<a id='step5'></a>

## 5. Calculate GCR

In this example we passed the parameter "pitch". Pitch is the spacing between rows (for example, between hub-posts) in a field.
To calculate Ground Coverage Ratio (GCR), we must relate the pitch to the collector-width by:
    
![GCR = CW / pitch](../images_wiki/Journal3Pics/Equation_GCR.png)

The collector width for our system must consider the number of panels and the y-gap:
    
![CW](../images_wiki/Journal3Pics/Equation_CW.png)
    
Collector Width gets saved in your module parameters (and later on your scene and trackerdict) as "sceney". You can calculate your collector width with the equation, or you can use this method to know your GCR:

In [6]:
# For more options on makemodule, see the help description of the function.  
CW = mymodule['sceney']
gcr = CW / pitch
print ("The GCR is :", gcr)

The GCR is : 0.4284


<a id='step6'></a>

## 6. Set Tracking Angles

This function will read the weather file, and based on the sun position it will calculate the angle the tracker should be at for each hour. It will create metdata files for each of the tracker angles considered.

For doing hourly simulations, remember to set **cumulativesky = False** here!

In [7]:
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)

C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:449: RuntimeWarning: invalid value encountered in arccos
  wc = np.degrees(np.arccos(temp))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:473: RuntimeWarning: invalid value encountered in arccos
  aoi = np.degrees(np.arccos(np.abs(np.sum(sun_vec*panel_norm, axis=0))))
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:568: RuntimeWarning: invalid value encountered in remainder
  surface_azimuth = surface_azimuth % 360
C:\ProgramData\Anaconda3\lib\site-packages\pvlib\tracking.py:572: RuntimeWarning: invalid value encountered in arccos
  surface_tilt = 90 - np.degrees(np.arccos(dotproduct))


In [8]:
print ("Full trackerdict for all the year created by set1axis: %s " % (len(demo.trackerdict))) 


Full trackerdict for all the year created by set1axis: 4245 


set1axis initializes the trackerdictionary Trackerdict. Trackerdict contains all hours in the year as keys. For example: trackerdict['12_16_08']. It is a return variable on many of the 1axis functions, but it is also stored inside of your Radiance Obj (i.e. demo.trackerdict). In this journal we are storing it as a variable to mute the option (otherwise it prints the returned trackerdict contents every time)


In [9]:
pprint.pprint(trackerdict['12_16_08'])


{'dhi': 18, 'ghi': 18, 'surf_azm': 90.0, 'surf_tilt': 4.61, 'theta': -4.61}


In [10]:
pprint.pprint(demo.trackerdict['12_16_08'])

{'dhi': 18, 'ghi': 18, 'surf_azm': 90.0, 'surf_tilt': 4.61, 'theta': -4.61}


All of the following functions add up elements to trackerdictionary to keep track (ba-dum tupzz) of the Scene and simulation parameters. In advanced journals we will explore the inner structure of trackerdict. For now, just now it exists :)

<a id='step7'></a>

## 7. Generate the Sky


We will create skies for each hour we want to model with the function gendaylit1axis. 

**If you don't specify a startdate and enddate, all the year will be created, which will take more time.**

For this example we are doing just two days in January. Format has to be a 'MM_DD' or 'MM/DD'

In [11]:
startdate = '01/13'     
enddate = '01/14'
trackerdict = demo.gendaylit1axis(startdate=startdate, enddate=enddate) 

Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 20 skyfiles in /skies/


Since we passed startdate and enddate to gendaylit, it will prune our trackerdict to only the desired days.
Let's explore our trackerdict:

In [12]:
print ("\nTrimmed trackerdict by gendaylit1axis to start and enddate length: %s " % (len(trackerdict)))
print ("")
trackerkeys = sorted(trackerdict.keys())
print ("Option of hours are: ", trackerkeys)
print ("")
print ("Contents of trackerdict for sample hour:")
pprint.pprint(trackerdict[trackerkeys[0]])


Trimmed trackerdict by gendaylit1axis to start and enddate length: 20 

Option of hours are:  ['01_13_08', '01_13_09', '01_13_10', '01_13_11', '01_13_12', '01_13_13', '01_13_14', '01_13_15', '01_13_16', '01_13_17', '01_14_08', '01_14_09', '01_14_10', '01_14_11', '01_14_12', '01_14_13', '01_14_14', '01_14_15', '01_14_16', '01_14_17']

Contents of trackerdict for sample hour:
{'dhi': 15,
 'ghi': 15,
 'skyfile': 'skies\\sky2_37.5_-77.33_01_13_08.rad',
 'surf_azm': 90.0,
 'surf_tilt': 3.62,
 'theta': -3.62}


<a id='step8'></a>

## 8. Make Scene 1axis

We can use gcr or pitch fo our scene dictionary.

In [13]:
# making the different scenes for the 1-axis tracking for the dates in trackerdict2.

sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows}  

trackerdict = demo.makeScene1axis(trackerdict = trackerdict, moduletype = moduletype, sceneDict = sceneDict) 


Making ~20 .rad files for gendaylit 1-axis workflow (this takes a minute..)
20 Radfiles created in /objects/


The scene parameteres are now stored in the trackerdict. To view them and to access them:
    

In [14]:
pprint.pprint(trackerdict[trackerkeys[0]])

{'clearance_height': 2.1647564917391366,
 'dhi': 15,
 'ghi': 15,
 'radfile': 'objects\\1axis01_13_08_2.164_10_3.62_20x7_origin0,0.rad',
 'scene': <bifacial_radiance.main.SceneObj object at 0x0000024A620952B0>,
 'skyfile': 'skies\\sky2_37.5_-77.33_01_13_08.rad',
 'surf_azm': 90.0,
 'surf_tilt': 3.62,
 'theta': -3.62}


In [15]:
pprint.pprint(demo.trackerdict[trackerkeys[5]]['scene'].__dict__)

{'bifi': 1,
 'gcr': 0.4284,
 'moduleDict': {'bifi': 1,
                'cellModule': {'numcellsx': 6,
                               'numcellsy': 12,
                               'xcell': 0.156,
                               'xcellgap': 0.02,
                               'ycell': 0.156,
                               'ycellgap': 0.02},
                'modulefile': 'objects\\Custom_Cell-Level_Module.rad',
                'numpanels': 2,
                'offsetfromaxis': 0,
                'scenex': 1.046,
                'sceney': 4.284,
                'scenez': 0.1,
                'text': '! genbox black cellPVmodule 0.156 0.156 0.02 | xform '
                        '-t -0.44 -2.142 0 -a 6 -t 0.176 0 0 -a 12 -t 0 0.176 '
                        '0 -a 2 -t 0 2.192 0\r\n'
                        '! genbox black octtube1a 1.046 0.04142135623730951 '
                        '0.1 | xform -t -0.523 -0.020710678118654756 '
                        '-0.15000000000000002\r\n'
          

<a id='step9a'></a>

## 9. Make Oct and Analyze 

### A. Make Oct and Analyze 1Hour

There are various options now to analyze the trackerdict hours we have defined. We will start by doing just one hour, because it's the fastest. Make sure to select an hour that exists in your trackerdict!

Options of hours:

In [16]:
pprint.pprint(trackerkeys)

['01_13_08',
 '01_13_09',
 '01_13_10',
 '01_13_11',
 '01_13_12',
 '01_13_13',
 '01_13_14',
 '01_13_15',
 '01_13_16',
 '01_13_17',
 '01_14_08',
 '01_14_09',
 '01_14_10',
 '01_14_11',
 '01_14_12',
 '01_14_13',
 '01_14_14',
 '01_14_15',
 '01_14_16',
 '01_14_17']


In [17]:
demo.makeOct1axis(singleindex='01_13_08')
results = demo.analysis1axis(singleindex='01_13_08')
print('\n\nHourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))


Making 1 octfiles in root directory.
Created 1axis_01_13_08.oct
Linescan in process: 1axis_01_13_08_Front
Linescan in process: 1axis_01_13_08_Back
Saved: results\irr_1axis_01_13_08.csv
Index: 01_13_08. Wm2Front: 14.201547962962962. Wm2Back: 2.0749953333333333

Saving Cumulative results
Saved: results\cumulative_results_.csv


Hourly bifi gain: 0.146


The trackerdict now contains information about the octfile, as well as the Analysis Object results

In [18]:
print ("\n Contents of trackerdict for sample hour after analysis1axis: ")
pprint.pprint(trackerdict[trackerkeys[0]])



 Contents of trackerdict for sample hour after analysis1axis: 
{'AnalysisObj': <bifacial_radiance.main.AnalysisObj object at 0x0000024A63358588>,
 'Wm2Back': [1.8268360000000001,
             1.75552,
             1.6788656666666666,
             1.60315,
             4.910752,
             1.6119296666666667,
             1.700885666666667,
             1.7575400000000003,
             1.829479],
 'Wm2Front': [14.864776666666666,
              14.865536666666666,
              14.866266666666666,
              14.866996666666667,
              8.873918333333334,
              14.869313333333332,
              14.869183333333334,
              14.86904,
              14.868900000000002],
 'backRatio': [0.12288870208149234,
               0.11808533751752538,
               0.11292362640072823,
               0.10782555551644596,
               0.5533292606823987,
               0.1083991729382972,
               0.11438229297778219,
               0.1181933606096554,
               0.

In [19]:
pprint.pprint(trackerdict[trackerkeys[0]]['AnalysisObj'].__dict__)

{'Wm2Back': [1.8268360000000001,
             1.75552,
             1.6788656666666666,
             1.60315,
             4.910752,
             1.6119296666666667,
             1.700885666666667,
             1.7575400000000003,
             1.829479],
 'Wm2Front': [14.864776666666666,
              14.865536666666666,
              14.866266666666666,
              14.866996666666667,
              8.873918333333334,
              14.869313333333332,
              14.869183333333334,
              14.86904,
              14.868900000000002],
 'backRatio': [0.12288870208149234,
               0.11808533751752538,
               0.11292362640072823,
               0.10782555551644596,
               0.5533292606823987,
               0.1083991729382972,
               0.11438229297778219,
               0.1181933606096554,
               0.12303236739991526],
 'mattype': ['a9.3.a2.2.0.cellPVmodule.6457',
             'a9.3.a2.4.0.cellPVmodule.6457',
             'a9.3.a2.7.0.cellPVmod

<a id='step9b'></a>

### B. Make Oct and Analye Range of Hours

You could do a range of indexes following a similar procedure:

In [20]:
for time in ['01_13_09','01_13_13']:  
    demo.makeOct1axis(singleindex=time)
    results=demo.analysis1axis(singleindex=time)

print('Accumulated hourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))


Making 1 octfiles in root directory.
Created 1axis_01_13_09.oct
Linescan in process: 1axis_01_13_09_Front
Linescan in process: 1axis_01_13_09_Back
Saved: results\irr_1axis_01_13_09.csv
Index: 01_13_09. Wm2Front: 97.91932592592592. Wm2Back: 12.036790925925926

Saving Cumulative results
Saved: results\cumulative_results_.csv

Making 1 octfiles in root directory.
Created 1axis_01_13_13.oct
Linescan in process: 1axis_01_13_13_Front
Linescan in process: 1axis_01_13_13_Back
Saved: results\irr_1axis_01_13_13.csv
Index: 01_13_13. Wm2Front: 422.10729629629634. Wm2Back: 38.627922222222224

Saving Cumulative results
Saved: results\cumulative_results_.csv
Accumulated hourly bifi gain: 0.0987


Note that the bifacial gain printed above is for the accumulated irradiance between the hours modeled so far. 
That is, demo.Wm2Back and demo.Wm2Front are for January 13, 8AM to 1 AM. Compare demo.Wm2back below with what we had before:

In [21]:
demo.Wm2Back

array([61.62960867, 56.09894433, 52.26810367, 47.526593  , 37.128162  ,
       49.24325267, 52.09034567, 56.446585  , 62.22578133])

To print the specific bifacial gain for a specific hour, you can use the following:

In [22]:
sum(trackerdict['01_13_13']['AnalysisObj'].Wm2Back) / sum(trackerdict['01_13_13']['AnalysisObj'].Wm2Front)

0.09151209316009438

<a id='step9c'></a>

### C. Make Oct and Analyze All Tracking Dictionary

This might considerably more time, depending on the number of entries on the trackerdictionary. If no **startdt** and **enddt** where specified on STEP **gendaylit1axis, this will run ALL of the hours in the year (~4000 hours).**


In [ ]:
demo.makeOct1axis()
results = demo.analysis1axis()
print('Accumulated hourly bifi gain for all the trackerdict: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))



Making 20 octfiles in root directory.
Created 1axis_01_13_08.oct
Created 1axis_01_13_09.oct
Created 1axis_01_13_10.oct
Created 1axis_01_13_11.oct
Created 1axis_01_13_12.oct
Created 1axis_01_13_13.oct
Created 1axis_01_13_14.oct
Created 1axis_01_13_15.oct
Created 1axis_01_13_16.oct
Created 1axis_01_13_17.oct
Created 1axis_01_14_08.oct
Created 1axis_01_14_09.oct
Created 1axis_01_14_10.oct
Created 1axis_01_14_11.oct
Created 1axis_01_14_12.oct
Created 1axis_01_14_13.oct
Created 1axis_01_14_14.oct
Created 1axis_01_14_15.oct
Created 1axis_01_14_16.oct
Created 1axis_01_14_17.oct
Linescan in process: 1axis_01_13_08_Front
Linescan in process: 1axis_01_13_08_Back
Saved: results\irr_1axis_01_13_08.csv
Index: 01_13_08. Wm2Front: 12.447584333333333. Wm2Back: 3.138846148148149
Linescan in process: 1axis_01_13_09_Front
Linescan in process: 1axis_01_13_09_Back
Saved: results\irr_1axis_01_13_09.csv
Index: 01_13_09. Wm2Front: 72.53930296296296. Wm2Back: 21.671116444444447
Linescan in process: 1axis_01_1

<div class="alert alert-warning">
Remember you should clean your results first! This will have torquetube and sky results if performed this way so don't trust this simplistic bifacial_gain examples.
</div>

<a id='condensed'></a>

### Condensed Version: All Tracking Dictionary

This is the summarized version to run gendaylit for one entries in the trackigndictionary.

In [ ]:
simulationName = 'Tutorial 3'
moduletype = 'Custom Cell-Level Module'    # We will define the parameters for this below in Step 4.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Tutorials\Journal2'
albedo = "litesoil"      # this is one of the options on ground.rad
lat = 37.5   
lon = -77.6

# Scene variables
nMods = 20
nRows = 7
hub_height = 2.3 # meters
pitch = 10 # meters      # We will be using pitch instead of GCR for this example.

# Traking parameters
cumulativesky = False
limit_angle = 45 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

#makeModule parameters
# x and y will be defined later on Step 4 for this tutorial!!
xgap = 0.01
ygap = 0.10
zgap = 0.05
numpanels = 2
torquetube = True
axisofrotationTorqueTube = False
diameter = 0.1
tubetype = 'Oct'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube of this material (0% reflectivity)

startdate = '11/06'     
enddate = '11/06'
demo = bifacial_radiance.RadianceObj(simulationName, path=testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat,lon) 
metdata = demo.readWeatherFile(epwfile)  
sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}  
demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
demo.gendaylit1axis(startdate=startdate, enddate=enddate)
demo.makeScene1axis(moduletype=moduletype,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
demo.makeOct1axis()
demo.analysis1axis()